# DataFrames with SQL


We talked about before that Spark is a unified computing engine. Among many things, it also means that Spark is not tied to one specific language. You can have access to the same transformation supported by the engine regardless if you're using the DF API or SQL.

When you express your logic in SQL, spark will compile the expression into an underlying execution plan built from primitives supported by the core engine. After that the job will be executed in the same way as we would have used the legacy spark API.

Let's see an example on how we can use SQL expressions with Dataframes.

In [1]:
import pyspark
from pyspark.sql import SparkSession

sc = pyspark.SparkContext()
spark = SparkSession(sc)

First, we need to create a dataframe in the same way we did in the previous module. We'll use the same sample dataset to create the DF with automated schema inference.

In [2]:
flightsDF = spark \
    .read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv('../data/flights.csv')

The next step is to turn the DF into a table or a view. This can be done with one simple method:

In [3]:
# create a View on top of the DF which can be used for querying with SQL
flightsDF.createOrReplaceTempView("flights")

In [4]:
res = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as total_destination
FROM flights
GROUP BY DEST_COUNTRY_NAME
ORDER BY total_destination DESC
LIMIT 5
""")
res.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|total_destination|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



As a comparison, let's see how the same expression would like with using the DataFrames API

In [5]:
from pyspark.sql.functions import desc
resDF = flightsDF \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "total_destination") \
    .sort(desc("total_destination")) \
    .limit(5)
resDF.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|total_destination|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



Let's compare the physical execution plan created by the two queries. We can see that the same transformations are triggered in the background.

In [6]:
res.explain()

resDF.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[total_destination#22L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#16,total_destination#22L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[sum(cast(count#18 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#16, 200), true, [id=#117]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[partial_sum(cast(count#18 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#16,count#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/andras/ipython_spark/spark_training_baseline/data/flights.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[total_destination#47L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#16,total_destination#47L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[sum(cast(count#18 as bigint))])
   +- Exchange has

## Managed SQL Tables and Databases

In [7]:
# spark.sql('DROP DATABASE test_db')
spark.sql('CREATE DATABASE test_db')
spark.sql('USE test_db')

DataFrame[]

Spark stores table metainformation in a Hive metastore which is by default located under the spark-warehouse subfolder

In [8]:
!ls -l spark-warehouse

total 0
drwxr-xr-x 1 andras andras 4096 Oct  5 11:48 test_db.db


In [9]:
flightsDF = spark \
    .read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv('../data/flights.csv')
flightsDF.write.saveAsTable('flights_managed')

In [11]:
spark.sql('SELECT * FROM flights_managed LIMIT 5').show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [12]:
spark.sql('drop table flights_managed')
spark.sql('drop database test_db')

DataFrame[]

If we're creating a Spark Managed Table as above, Spark not just managing the table metadata but the data itself as well. If you drop a managed table, the data will be also deleted.

### Unmanaged tables

In [14]:
spark.sql('create database test_db')
spark.sql("CREATE TABLE flights_unmanaged(DEST_COUNTRY_NAME string, ORIG_COUNTRY_NAME string, count int) using csv options (path '/tmp/flights.csv')")

DataFrame[]

In [15]:
spark.sql('select * from flights_unmanaged limit 10').show()

+-----------------+-----------------+-----+
|DEST_COUNTRY_NAME|ORIG_COUNTRY_NAME|count|
+-----------------+-----------------+-----+
+-----------------+-----------------+-----+



In [19]:
flightsDF \
    .write \
    .option('path', '/tmp/flights.csv') \
    .insertInto('flights_unmanaged')

In [20]:
spark.sql('select * from flights_unmanaged limit 10').show()

+-----------------+-----------------+-----+
|DEST_COUNTRY_NAME|ORIG_COUNTRY_NAME|count|
+-----------------+-----------------+-----+
|    United States|          Romania|   15|
|    United States|          Croatia|    1|
|    United States|          Ireland|  344|
|            Egypt|    United States|   15|
|    United States|            India|   62|
|    United States|        Singapore|    1|
|    United States|          Grenada|   62|
|       Costa Rica|    United States|  588|
|          Senegal|    United States|   40|
|          Moldova|    United States|    1|
+-----------------+-----------------+-----+

